In [ ]:
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import generate_inputs as gi
import file_methods as fm
import pandas as pd
import training_tools as tt

import yaml

import wandb
from wandb.keras import WandbCallback


print(tf.__version__)

In [ ]:
# wandb configuration
wandb.init(project="pH-box-model")

# hyperparameters
wandb.config.dropout = 0.2
wandb.config.learning_rate = 0.1
wandb.config.epochs = 100
wandb.config.batch_size = 50

In [ ]:
# Unpickle the saved data set.
test_set = fm.unpickle('test.pickle')
train_set = fm.unpickle('train.pickle')
val_set = fm.unpickle('val.pickle')

In [ ]:
# Put data into a dataframe for visualisation.
vis = pd.DataFrame()

for i in train_set:
    append = pd.DataFrame.from_dict(train_set[i].condition_blocks['Seawater'].primary_species)
    vis = vis.append(append, ignore_index = True)
    
# Create series from output data (pH).
pH = pd.DataFrame()

for i in train_set:
    pH = pH.append(train_set[i].results.results_dict['pH']['pH'], ignore_index = True)

pH.columns = ['pH']

# Normalise the pH to be on the scale 0-1ish
pH['pH'] = pH['pH'] / 14

x_train = vis.to_numpy()
y_train = pH.to_numpy()

print(np.shape(x_train))

In [ ]:
# Put data into a dataframe for visualisation.
vis = pd.DataFrame()

for i in test_set:
    append = pd.DataFrame.from_dict(test_set[i].condition_blocks['Seawater'].primary_species)
    vis = vis.append(append, ignore_index = True)
    
# Create series from output data (pH).
pH = pd.DataFrame()

for i in test_set:
    pH = pH.append(test_set[i].results.results_dict['pH']['pH'], ignore_index = True)

pH.columns = ['pH']

# Normalise the pH to be on the scale 0-1ish
pH['pH'] = pH['pH'] / 14

x_test = vis.to_numpy()
y_test = pH.to_numpy()

In [ ]:
with open('sweep.yaml') as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    sweep_config = yaml.load(file, Loader=yaml.FullLoader)

sweep_config

In [ ]:
# The following variables are the hyperparameters.
learning_rate = wandb.config.learning_rate
epochs = wandb.config.epochs
batch_size = wandb.config.batch_size
dropout =  wandb.config.dropout 

# Establish the model's topography.
model = tt.create_model(learning_rate, dropout)

In [ ]:
# Train the model on the normalized training set. We're passing the entire
# normalized training set, but the model will only use the features
# defined by the feature_layer.
history = tt.train_model(model, x_train, y_train, epochs, batch_size)
tt.plot_history(history)

# After building a model against the training set, test that model
# against the test set.
test_features = x_test
test_label = y_test
print("\n Evaluate the new model against the test set:")
model.evaluate(x = x_test, y = y_test, batch_size=batch_size)

In [15]:
def train():
    # wandb configuration
    wandb.init(project="pH-box-model")

    # Hyperparameters defaults.
    wandb.config.dropout = 0.2
    wandb.config.learning_rate = 0.1
    wandb.config.epochs = 100
    wandb.config.batch_size = 50
    
    # Assign hyperparameters.


    # Define the model geometry.
    model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=0.1)),
    tf.keras.layers.Dropout(wandb.config.dropout),
    tf.keras.layers.Dense(10)
    ])

    # Compile the model.
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=wandb.config.learning_rate),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mse'])

    # Split the dataset into features and label.
    model.fit(x=x_train, y=y_train, batch_size=wandb.config.batch_size, epochs=wandb.config.epochs, validation_split = 0.1, shuffle=True, verbose=0, callbacks=[WandbCallback()])
    
    %%wandb 

In [16]:
sweep_id = wandb.sweep(sweep_config)

Create sweep with ID: 7r66ww0f
Sweep URL: https://app.wandb.ai/a-fotherby/pH-box-model/sweeps/7r66ww0f


In [17]:
wandb.agent(sweep_id, function=train) 

INFO:wandb.wandb_agent:Running runs: []
INFO:wandb.wandb_agent:Agent received command: run
INFO:wandb.wandb_agent:Agent starting run with config:
	batch_size: 49
	dropout: 0.2058592019120478
	epochs: 70
	learning_rate: 0.09587909058949781


wandb: Agent Starting Run: jabk39b0 with config:
	batch_size: 49
	dropout: 0.2058592019120478
	epochs: 70
	learning_rate: 0.09587909058949781


INFO:wandb.wandb_agent:Running runs: ['jabk39b0']
wandb: Ctrl-c pressed. Waiting for runs to end. Press ctrl-c again to terminate them.


Error in callback <function _init_jupyter.<locals>.cleanup at 0x14d2a80d0> (for post_run_cell):


KeyboardInterrupt: 

In [14]:
train()


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

